# NOTEBOOK QWEN 2.5 - MULTICLASS CLASSIFICATION (FINE-TUNING) WITH LESS SAMPLES

This notebook evaluates the Qwen LLM using fine-tuning prompting for multiclass classification, distinguishing between normal or the different types of attack **with less samples**

## 1. Objectives

The objective of this notebook is to evaluate the performance of the Qwen LLM in a **fine-tuned multiclass classification** setting.

Specifically, this notebook aims to:
- Assess the model's ability to distinguish between **normal** and the **different types of attack** with prior examples.
- Analyze its predictions and evaluate its suitability for multiclass threat detection tasks in IoT systems.

## 2. IMPORTS AND SETUP
Import the required python libraries, preprocessing, training (fine-tuning) and evaluation.

In [1]:
# pandas, json, re and os imports
import pandas as pd
import json
import os
import re
# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
# imports for LLM training
from datasets import Dataset, DatasetDict
from huggingface_hub import interpreter_login
from transformers import (
    TrainingArguments,
    EarlyStoppingCallback
)
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
from collections import defaultdict
# imports for evaluation (sklearn)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# tqdm import
from tqdm import tqdm

# Login to Hugging Face Hub
interpreter_login()

/home/jorge/TFM/threatlogllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



## 3. PREPARE DATASET

The dataset is split into 70% for training, 15% for validation, and 15% for testing. The model is fine-tuned on the training set while using the validation set to monitor performance during training. After fine-tuning, the final evaluation is performed on the test set.

In [ ]:
# Define file path
file_path = "../../../data/prompts/multiclass_instructions_for_inference_less_samples.jsonl"

# Check if file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist.")

# Read JSONL file line by line
samples = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line.strip())  # Load each line as a JSON object
        samples.append({"text": sample["Prompt"]})  # Extract the prompt field

# Convert into Hugging Face Dataset format
dataset_dict = {"full": Dataset.from_list(samples)}
dataset = DatasetDict(dataset_dict)

# Verify first example
print(dataset["full"][0])

{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n### Instruction:\nYou are a cybersecurity expert specializing in IoT security. Your task is to analyze network logs and determine whether the given log data indicates a potential attack. Only provide the type of attack if it is an attack. If the log data is normal traffic, state that it is normal traffic.\n### Question:\n- The length of the DNS query is: 0.0\n- The MQTT protocol name used is: 0.0\n- The MQTT message type is: 0.0\n- The MQTT topic is: 0.0\n- The MQTT connection acknowledgment flags are: 0.0\n- TCP options set in the packet are: 0101080a3df2b58a2b88a4a5\n- TCP destination port is: 60210.0\n### Response:\n<think>\nThis log data is an attack!! The type of attack detecte

In [3]:
# Shuffle the dataset
dataset = dataset["full"].shuffle(seed=42)

# Split dataset into train (70%), validation (15%), test (15%)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))

# Ensure the split sizes are correct
train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, train_size + val_size))
test_dataset = dataset.select(range(train_size + val_size, len(dataset)))

# Print sizes
print(f"Train Samples: {len(train_dataset)}")
print(f"Validation Samples: {len(val_dataset)}")
print(f"Test Samples: {len(test_dataset)}")

Train Samples: 5250
Validation Samples: 1125
Test Samples: 1125


## 4. PREPARE LLM

Load the Qwen language model from Unsloth with specific configuration parameters for fine-tuning.

In [4]:
# Define model loading parameters
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

# Load the Qwen LLM with Unsloth's FastLanguageModel utility
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4060 Laptop GPU. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
# Get the model's config
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj",
        "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",
    use_gradient_checkpointing="unsloth",  
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Reach the model for training
FastLanguageModel.for_training(model)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

## 5. PREPARE DATASET WITH EOS

In this section, each sample in the dataset is formatted to include an end-of-sentence (EOS) token.  
This ensures proper termination of sequences during training and improves model understanding of prompt boundaries.

In [6]:
EOS_TOKEN = tokenizer.eos_token  # Ensure each training sample has an EOS token

# Formatting function to structure data properly
def formatting_prompts_func(examples):
    texts = []
    for entry in examples["text"]:  # "text" is the key in our dataset
        text = entry + EOS_TOKEN  # Append EOS token
        texts.append(text)
    return {"text": texts}

# Apply formatting
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)

# Print an example formatted sample
print(train_dataset["text"][0])

Map: 100%|██████████| 1125/1125 [00:00<00:00, 73856.09 examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
### Instruction:
You are a cybersecurity expert specializing in IoT security. Your task is to analyze network logs and determine whether the given log data indicates a potential attack. Only provide the type of attack if it is an attack. If the log data is normal traffic, state that it is normal traffic.
### Question:
- The length of the DNS query is: 0.0
- The MQTT protocol name used is: 0.0
- The MQTT message type is: 0.0
- The MQTT topic is: 0.0
- The MQTT connection acknowledgment flags are: 0.0
- TCP options set in the packet are: 020405b40402080a0aa7dd390000000001030307
- TCP destination port is: 80.0
### Response:
<think>
This log data is an attack!! The type of attack detected is DDoS_

## 6. INITIALIZE THE SFTTrainer FOR FINE-TUNING

This section initializes the `SFTTrainer` with the model, tokenizer, and prepared datasets.  
It sets training arguments such as learning rate, batch size, evaluation strategy, and checkpointing.  
The model will be fine-tuned using supervised learning and validated on the evaluation set.

In [7]:
# Prepare the trainer, with SFTTrainer and TrainingArguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs_qwen",
        eval_strategy="steps",  
        save_strategy="steps",  
        save_total_limit=2,  
        metric_for_best_model="eval_loss",  
        load_best_model_at_end=True,  
        greater_is_better=False,  
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stop if loss doesn't improve in 3 evaluations
)

Map (num_proc=2): 100%|██████████| 1125/1125 [00:01<00:00, 998.91 examples/s]


## 7. TRAINING MODEL

In this section, the model is fine-tuned for up to 1000 steps.  
Evaluation is performed on the validation set every 10 steps, and early stopping is applied if no improvement is observed for 3 consecutive evaluations.

In [8]:
# Start fine-tuning
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,250 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss,Validation Loss
10,2.151900,0.941601
20,0.396900,0.177934
30,0.141700,0.146417
40,0.132200,0.138662
50,0.135000,0.134454
60,0.122000,0.133342
70,0.121000,0.133185
80,0.126900,0.127502
90,0.116900,0.124652
100,0.113200,0.121917


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Could not locate the best model at outputs_qwen/checkpoint-400/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


In [9]:
# Save fine-tuned model and tokenizer
trainer.save_model("fine-tuned-model-attacks/qwen_finetuned_model_multi")  # Saves model weights
tokenizer.save_pretrained("fine-tuned-model-attacks/qwen_finetuned_model_multi")  # Saves tokenizer config

('fine-tuned-model-attacks/qwen_finetuned_model_multi/tokenizer_config.json',
 'fine-tuned-model-attacks/qwen_finetuned_model_multi/special_tokens_map.json',
 'fine-tuned-model-attacks/qwen_finetuned_model_multi/vocab.json',
 'fine-tuned-model-attacks/qwen_finetuned_model_multi/merges.txt',
 'fine-tuned-model-attacks/qwen_finetuned_model_multi/added_tokens.json',
 'fine-tuned-model-attacks/qwen_finetuned_model_multi/tokenizer.json')

## 8. COMPUTE PREDICTIONS

In this section, predictions are generated using the LLM (`predictions_multiclass`) and stored in a list. The corresponding true labels are also collected in a separate list (`actual_labels_multiclass`). 

Both lists will be used later to compute evaluation metrics.

In [10]:
# Ensures optimized inference
FastLanguageModel.for_inference(model)  

# Initialize lists to store actual vs predicted labels
predictions_multiclass = []
actual_labels_multiclass = []

# Define attack types for mapping
attack_types = [
    "Normal", "DDoS_UDP", "DDoS_ICMP", "SQL_injection", "Password",
    "Vulnerability_scanner", "DDoS_TCP", "DDoS_HTTP", "Uploading", "Backdoor",
    "Port_Scanning", "XSS", "Ransomware", "MITM", "Fingerprinting"
]

# Create mapping functions
attack_type_to_id = {attack: i for i, attack in enumerate(attack_types)}
id_to_attack_type = {i: attack for i, attack in enumerate(attack_types)}

def label_to_multiclass(text):
    """Convert response text to a multi-class attack label (integer ID)."""
    for attack in attack_types:
        if attack in text:
            return attack_type_to_id[attack]  # Return attack ID
    return attack_type_to_id["Normal"]  # Default to Normal if no attack detected

# Iterate over test dataset
for sample in tqdm(test_dataset, desc="Processing test samples", unit="sample"):
    full_text = sample["text"]

    # Verifiy if the text contains "### Response:"
    if "### Response:" in full_text:
        input_text, actual_label = full_text.split("### Response:")
        input_text = input_text.strip()  # Remove extra spaces
        actual_label = label_to_multiclass(actual_label.strip())  # Convert actual label to ID
    else:
        input_text = full_text.strip()
        actual_label = attack_type_to_id["Normal"]  # Default to "Normal" if missing

    # Tokenize input
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda")

    # Generate response
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=200,
        use_cache=True,
    )

    # Decode prediction and convert to attack type ID
    predicted_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    if "### Response:" in predicted_response:
        predicted_label = label_to_multiclass(predicted_response.split("### Response:")[1].strip())
    else:
        predicted_label = attack_type_to_id["Normal"]  # Default if response format is incorrect

    # Store results
    predictions_multiclass.append(predicted_label)
    actual_labels_multiclass.append(actual_label)

Processing test samples: 100%|██████████| 1125/1125 [36:16<00:00,  1.93s/sample]


## 9. Compute Evaluation Metrics

In this section, standard evaluation metrics are computed to assess the model's performance. These include Accuracy, Precision, Recall, F1-Score, and the Confusion Matrix.

### GENERAL METRICS

In [11]:
# Compute Multi-Class Evaluation Metrics
accuracy = accuracy_score(actual_labels_multiclass, predictions_multiclass)
precision = precision_score(actual_labels_multiclass, predictions_multiclass, average="weighted", zero_division=0)
recall = recall_score(actual_labels_multiclass, predictions_multiclass, average="weighted", zero_division=0)
f1 = f1_score(actual_labels_multiclass, predictions_multiclass, average="weighted", zero_division=0)

# Print results
print("\nMulti-Class Evaluation General Results:")
print(f" -Accuracy: {accuracy:.4f}")
print(f" -Precision (Weighted): {precision:.4f}")
print(f" -Recall (Weighted): {recall:.4f}")
print(f" -F1-score (Weighted): {f1:.4f}")


Multi-Class Evaluation General Results:
 -Accuracy: 0.4756
 -Precision (Weighted): 0.5974
 -Recall (Weighted): 0.4756
 -F1-score (Weighted): 0.4546


### METRICS PER CLASS

In [12]:
# Compute Per-Class Metrics
class_metrics = defaultdict(dict)
for attack_id in set(actual_labels_multiclass):
    attack_name = id_to_attack_type[attack_id]

    if attack_id in predictions_multiclass:
        class_metrics[attack_name]["Precision"] = precision_score(actual_labels_multiclass, predictions_multiclass, labels=[attack_id], average="macro", zero_division=0)
        class_metrics[attack_name]["Recall"] = recall_score(actual_labels_multiclass, predictions_multiclass, labels=[attack_id], average="macro", zero_division=0)
        class_metrics[attack_name]["F1-score"] = f1_score(actual_labels_multiclass, predictions_multiclass, labels=[attack_id], average="macro", zero_division=0)
    else:
        class_metrics[attack_name] = {"Precision": 0, "Recall": 0, "F1-score": 0}  # Set default values if no predictions

# Print per-class metrics
print("\n\nPer-Class Performance:")
for attack, metrics in class_metrics.items():
    print(f"{attack:25s} | Precision: {metrics['Precision']:.4f} | Recall: {metrics['Recall']:.4f} | F1-score: {metrics['F1-score']:.4f}")



Per-Class Performance:
Normal                    | Precision: 1.0000 | Recall: 1.0000 | F1-score: 1.0000
DDoS_UDP                  | Precision: 0.2646 | Recall: 0.7353 | F1-score: 0.3891
DDoS_ICMP                 | Precision: 0.2875 | Recall: 0.3108 | F1-score: 0.2987
SQL_injection             | Precision: 0.3182 | Recall: 0.1061 | F1-score: 0.1591
Password                  | Precision: 0.6415 | Recall: 0.4533 | F1-score: 0.5312
Vulnerability_scanner     | Precision: 0.3200 | Recall: 0.6667 | F1-score: 0.4324
DDoS_TCP                  | Precision: 0.5379 | Recall: 0.9342 | F1-score: 0.6827
DDoS_HTTP                 | Precision: 0.2745 | Recall: 0.5250 | F1-score: 0.3605
Uploading                 | Precision: 0.5897 | Recall: 0.3382 | F1-score: 0.4299
Backdoor                  | Precision: 0.6731 | Recall: 0.8642 | F1-score: 0.7568
Port_Scanning             | Precision: 0.8333 | Recall: 0.1639 | F1-score: 0.2740
XSS                       | Precision: 0.5217 | Recall: 0.1429 | F1-score

In [13]:
# Convert numeric labels back to attack type names
actual_labels_str = [id_to_attack_type[label] for label in actual_labels_multiclass]
predictions_str = [id_to_attack_type[label] for label in predictions_multiclass]

# Compute the confusion matrix
cm = confusion_matrix(actual_labels_str, predictions_str, labels=attack_types)

# Create a DataFrame for better visualization
cm_df = pd.DataFrame(cm, index=attack_types, columns=attack_types)

# Plot the confusion matrix using Seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', linewidths=0.5)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Qwen Multiclass fine-tuned")
plt.xticks(rotation=45, ha="right")  # Rotate labels for better readability
plt.yticks(rotation=0)

# Show the plot
plt.show()